# Preparação

In [2]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00


In [3]:
import wandb

In [4]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
rm -rf `find -type d -name .ipynb_checkpoints`

In [7]:
# Import packages and set numpy random seed
import numpy as np
np.random.seed(5)
import tensorflow as tf
tf.random.set_seed(2)
from datasets import sign_language
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
# Load pre-shuffled training and test datasets
(x_train, y_train), (x_test, y_test) = sign_language.load_data()

# Store labels of dataset
#labels = ['A', 'B', 'C']

# Visualização

In [ ]:
import random

# Launch 5 simulated experiments
for run in range(5):
    # 1️⃣ Start a new run to track this script
    wandb.init(
        # Set entity to specify your username or team name
        # ex: entity="carey",
        # Set the project where this run will be logged
        project="basic-intro",
        # Track hyperparameters and run metadata
        config={
            "learning_rate": 0.02,
            "architecture": "CNN",
            "dataset": "CIFAR-100",
        },
    )

    # This simple block simulates a training loop logging metrics
    offset = random.random() / 5
    for ii in range(2, 10):
        acc = 1 - 2**-ii - random.random() / ii - offset
        loss = 2**-ii + random.random() / ii + offset
        # 2️⃣ Log metrics from your script to W&B
        wandb.log({"acc": acc, "loss": loss})

    # Mark the run as finished
    wandb.finish()

In [ ]:
# Print the first several training images, along with the labels
fig = plt.figure(figsize=(20,5))
for i in range(36):
    ax = fig.add_subplot(3, 12, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_train[i]))
    ax.set_title("{}".format(labels[y_train[i]]))
plt.show()

In [ ]:
# Number of A's in the training dataset
num_A_train = sum(y_train==0)
# Number of B's in the training dataset
num_B_train = sum(y_train==1)
# Number of C's in the training dataset
num_C_train = sum(y_train==2)

# Number of A's in the test dataset
num_A_test = sum(y_test==0)
# Number of B's in the test dataset
num_B_test = sum(y_test==1)
# Number of C's in the test dataset
num_C_test = sum(y_test==2)

# Print statistics about the dataset
print("Training set:")
print("\tA: {}, B: {}, C: {}".format(num_A_train, num_B_train, num_C_train))
print("Test set:")
print("\tA: {}, B: {}, C: {}".format(num_A_test, num_B_test, num_C_test))

# Treino com Wandb

In [45]:
import random
import numpy as np
import tensorflow as tf
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

# Simple Keras Model

# Launch 5 experiments, trying different dropout rates
for run in range(5):
    # Start a run, tracking hyperparameters
    wandb.init(
        project="keras-intro",
        # (optional) set entity to specify your username or team name
        # entity="my_team",
        config={
            "layer_1": 512,
            "activation_1": "relu",
            "dropout": random.uniform(0.01, 0.80),
            "layer_2": 10,
            "activation_2": "softmax",
            "optimizer": "sgd",
            "loss": "sparse_categorical_crossentropy",
            "metric": "accuracy",
            "epoch": 6,
            "batch_size": 256,
        },
    )
    config = wandb.config

    # Get the data
    (x_train, y_train), (x_test, y_test) = sign_language.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train, y_train = x_train[::5], y_train[::5]  # Subset data for a faster demo
    x_test, y_test = x_test[::20], y_test[::20]
    labels = [str(digit) for digit in range(np.max(y_train) + 1)]

    # Build a model
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Conv2D(11, (4,4), activation=tf.nn.relu, input_shape=(50, 50, 3)),
            tf.keras.layers.Dense(config.layer_1, activation=config.activation_1),
            tf.keras.layers.Dropout(config.dropout),
            tf.keras.layers.Dense(config.layer_2, activation=config.activation_2),
            tf.keras.layers.Conv2D(11, (4,4), activation=tf.nn.relu),
            tf.keras.layers.GlobalAveragePooling2D(),
        ]
    )

    model.compile(optimizer=config.optimizer, loss=config.loss, metrics=[config.metric])

    # Add WandbMetricsLogger to log metrics and WandbModelCheckpoint to log model checkpoints
    wandb_callbacks = [
        WandbMetricsLogger(),
        WandbModelCheckpoint(filepath="my_model_{epoch:02d}"),
    ]

    model.fit(
        x=x_train,
        y=y_train,
        epochs=config.epoch,
        batch_size=config.batch_size,
        validation_data=(x_test, y_test),
        callbacks=wandb_callbacks,
    )

    # Mark the run as finished
    wandb.finish()

Epoch 1/6
2/2 [==============================] - ETA: 0s - loss: 2.9005 - accuracy: 0.0594    

wandb: Adding directory to artifact (./my_model_01)... Done. 0.0s


2/2 [==============================] - 12s 3s/step - loss: 2.9005 - accuracy: 0.0594 - val_loss: 1.1523 - val_accuracy: 0.4000
Epoch 2/6
2/2 [==============================] - ETA: 0s - loss: 1.3081 - accuracy: 0.3594

wandb: Adding directory to artifact (./my_model_02)... Done. 0.0s


2/2 [==============================] - 12s 3s/step - loss: 1.3081 - accuracy: 0.3594 - val_loss: 1.0952 - val_accuracy: 0.4000
Epoch 3/6
2/2 [==============================] - ETA: 0s - loss: 1.2278 - accuracy: 0.3594

...

wandb: Adding directory to artifact (./my_model_05)... Done. 0.0s


2/2 [==============================] - 12s 3s/step - loss: 1.1719 - accuracy: 0.3594 - val_loss: 1.0596 - val_accuracy: 0.4000
Epoch 6/6
2/2 [==============================] - ETA: 0s - loss: 1.1591 - accuracy: 0.3594

wandb: Adding directory to artifact (./my_model_06)... Done. 0.0s


2/2 [==============================] - 12s 4s/step - loss: 1.1591 - accuracy: 0.3594 - val_loss: 1.0573 - val_accuracy: 0.4000


epoch/accuracy,▁█████
epoch/epoch,▁▂▄▅▇█
epoch/learning_rate,▁▁▁▁▁▁
epoch/loss,█▂▁▁▁▁
epoch/val_accuracy,▁▁▁▁▁▁
epoch/val_loss,█▄▂▁▁▁
epoch/accuracy,0.35938
epoch/epoch,5
epoch/learning_rate,0.01
epoch/loss,1.15909
epoch/val_accuracy,0.4


# Treino normal

In [ ]:
from keras.utils import np_utils

# One-hot encode the training labels
y_train_OH = np_utils.to_categorical(y_train)

# One-hot encode the test labels
y_test_OH = np_utils.to_categorical(y_test)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense
from keras.models import Sequential

model = Sequential()
# First convolutional layer accepts image input
model.add(Conv2D(filters=5, kernel_size=5, padding='same', activation='relu',
                        input_shape=(50, 50, 3)))
# Add a max pooling layer
model.add(MaxPooling2D(pool_size=(4, 4)))
# Add a convolutional layer
model.add(Conv2D(filters=15, kernel_size=5, padding='same', activation='relu'))
# Add another max pooling layer
model.add(MaxPooling2D(pool_size=(4, 4)))
# Flatten and feed to output layer
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

# Summarize the model
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
hist = model.fit(x=x_train, y=y_train_OH, validation_split=0.20, epochs=2, batch_size=32)

In [ ]:
# Obtain accuracy on test set
score = model.evaluate(x=x_test,
                       y=y_test_OH,
                       verbose=0)
print('Test accuracy:', score[1])

In [ ]:
# Get predicted probabilities for test dataset
y_probs = model.predict(x_test)

# Get predicted labels for test dataset
y_preds = np.argmax(y_probs, axis=1)

# Indices corresponding to test images which were mislabeled
bad_test_idxs = np.where(y_preds!=y_test)[0]

# Print mislabeled examples
fig = plt.figure(figsize=(25,4))
for i, idx in enumerate(bad_test_idxs):
    ax = fig.add_subplot(2, np.ceil(len(bad_test_idxs)/2), i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[idx]))
    ax.set_title("{} (pred: {})".format(labels[y_test[idx]], labels[y_preds[idx]]))